In [1]:
import torch
import numpy as np
import pandas as pd
import math
import dgl
import pickle
from multisage.builder import PandasGraphBuilder
import ast

In [2]:
df = pd.read_excel('data.xlsx')
df = df[df['상품 카테고리 중분류'] != '향수'].reset_index(drop=True)

In [3]:
for i in range(len(df)):
    df['특징'][i] = ast.literal_eval(df['특징'][i])
    new_features = []
    for j in df['특징'][i]:
        if ', ' in j:
            new_features.extend(j.split(', '))
            for k in j.split(', '):
                if ',' in k:
                    new_features.extend(k.split(','))
        elif ',' in j:
            new_features.extend(j.split(','))
            for k in j.split(','):
                if ', ' in k:
                    new_features.extend(k.split(', '))
        else:
            new_features.append(j)
    df.at[i, '특징'] = new_features

for i in range(len(df)):
    df['성분'][i] = ast.literal_eval(df['성분'][i])

for i in range(len(df)):
    df['느낌'][i] = ast.literal_eval(df['느낌'][i])

C:\Users\kimhw\AppData\Local\Temp\ipykernel_148\792861745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['특징'][i] = ast.literal_eval(df['특징'][i])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_148\792861745.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['성분'][i] = ast.literal_eval(df['성분'][i])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_148\792861745.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['느낌'][i] = ast.l

In [4]:
df['특징'][3024]

['핸드케어', '모든피부용', '향', '촉촉함(수분공급)', '흡수력', '발림성', '튜브형']

In [5]:
df['특징'][3032]

['핸드케어', '모든피부용', '기타향', '향', '촉촉함(수분공급)', '펌프형']

In [6]:
def graph_maker(df):
        
    attributes_df, attributes, attributes_merged_df = seperateDF(df, '특징')
    ingredient_df, ingredients, ingredient_merged_df = seperateDF(df,'성분')
    feel_df, feels, feel_merged_df = seperateDF(df,'느낌')

    graph_builder = PandasGraphBuilder()
    graph_builder.add_entities(attributes, '특징_id', '특징')
    graph_builder.add_entities(df, 'ID', '상품')
    graph_builder.add_binary_relations(attributes_merged_df, '특징_id', 'ID', 'define')
    graph_builder.add_binary_relations(attributes_merged_df, 'ID', '특징_id', 'define-by')
    g = graph_builder.build()

    g.nodes['특징'].data['id'] = torch.LongTensor(attributes['특징_id'].cat.codes.values)

    attribute_columns = attributes_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    attributes_df[attribute_columns] = attributes_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['특징'] = torch.FloatTensor(attributes_df[attribute_columns].values)

    attribute_columns = ingredient_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    ingredient_df[attribute_columns] = ingredient_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['성분'] = torch.FloatTensor(ingredient_df[attribute_columns].values)

    attribute_columns = feel_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    feel_df[attribute_columns] = feel_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['느낌'] = torch.FloatTensor(feel_df[attribute_columns].values)

    g.edges['define'].data['rating'] = torch.LongTensor(df['평점'].values)
    g.edges['define-by'].data['rating'] = torch.LongTensor(df['평점'].values)

    # 그래프 구조 확인

    print("Number of nodes per type:")
    print(g.number_of_nodes())
    print("Number of edges per type:")
    print(g.number_of_edges())
    print("Node types:")
    print(g.ntypes)
    print("Edge types:")
    print(g.etypes)

    # 노드 데이터 확인
    for ntype in g.ntypes:
        print(f"Node type: {ntype}")
        print(g.nodes[ntype].data)

    # 엣지 데이터 확인
    for stype, etype, dtype in g.canonical_etypes:
        print(f"Edge type: {etype}")
        print(g.edges[etype].data)

    # 그래프 구조 확인
    print("Graph structure:")
    print(g)

    return g

In [7]:
def seperateDF(df, name):
    df_copy = df.copy()
    
    small_df = df_copy.explode(name)[['ID','상품명',name,'평점']]
    attributes = pd.DataFrame(small_df[name].unique()).reset_index()
    attributes.columns = [name + '_id', name]
    attributes = pd.DataFrame(attributes).astype({name + '_id': 'category'})
    attribute_list = list(attributes[name])
    merged_df = pd.merge(small_df, attributes, on=[name])

    for i in attribute_list:
        data_list = []
        print(i)
        for j in range(len(df)):
            if i in df[name][j]:
                data_list.append(True)
            else:
                data_list.append(False)
        df_copy = pd.concat([df_copy, pd.DataFrame({i: data_list})], axis=1)

    return df_copy, attributes, merged_df

In [ ]:
g = graph_maker(df)

output_path = 'dataset/Beauty/beauty_graph.dgl'
dgl.save_graphs(output_path, [g])

dataset = {
    'train-graph': g,
    'context-type': '특징',
    'item-type': '상품',
    'context-to-item-type': 'define',
    'item-to-context-type': 'define-by'}

with open('dataset/Beauty/beauty_data.pickle', 'wb') as f:
    pickle.dump(dataset, f)

선스틱
페이스용
아이용
바디용
모든피부용
SPF50+
PA++++
부드러운 발림
백탁현상방지
촉촉함(수분공급)
저자극
지속력
밀착력
어린이겸용
선스프레이
PA+++
샴푸
일반샴푸
펌프형
세정력
상쾌함
각질케어
청량감
실리콘 프리
시트러스향
아로마향
두피샴푸
비듬케어
모발영양
기타향
모든두피용
쿨샴푸
풍부한 거품
계면활성제 프리
마스크시트
피부진정
피부탄력
영양공급
시트형
에센스
원터치형
패치형
바디클렌저
그린향
향
쿨링감
선파우더/쿠션
50+
피부톤보정
흡수력
리미티드에디션
두피케어
두피토닉
두피진정
탈모샴푸
천연샴푸
비듬샴푸
머릿결개선
유분제거
약산성
손상케어
모든 모발용
파라벤 프리
필오프팩
피지케어
피부결정돈
트러블케어
미스트
파우치형
건성
중건성
복합
지복합
지성
유기자차
노폐물 제거
분사력
레포츠용
워터프루프
무스형
선크림
혼합자차(유기+무기)
50
샴푸바
고체형
모근강화
탈모케어
튜브형
무기자차
티트리
어성초
히알루론산
브라이트닝
리퀴드형
두피스케일러
민감성
PA++
스킨/토너
한국비건인증원
화이트닝
세럼
젤형
오일 프리
볼륨효과
쉐도우형
여드름성완화도움
산성샴푸
뚜껑형
워시오프팩
클렌징비누
넥
무향
팔자주름
이브비건
모발윤기
액상형
탈모증상완화도움
낮
밤
유수분밸런스조절
수분미스트
안개분사
두피에센스
스프레이형
SPF50
모공케어
윤기부여
크림
유향
앰플
피부강화
비건소사이어티
청소년겸용
크림형
안티에이징
주름케어
스틱형
새치케어샴푸
리필형
스포이드형
거품형
핸드케어
피부투명
플로럴향
과일향
콜라겐
임산부겸용
피지조절
가는 모발용
손상 모발용
우디향
미백도움
오일형
주름개선도움
퍼퓸샴푸
피부보호
머스크향
알로에
롤온형
고보습
발림성
주니어겸용
보색샴푸
핸드워시
베이비파우더향
항균
밤형
핸드크림
로션형
퍼스트에센스
시어버터
메이크업베이스 겸용
스팟
스팟케어
아르간
세라마이드
유아겸용
핸드밤
코튼향
핸드로션
캡형
오리엔탈향
아보카도
핸드마스크팩
시트
패드형
글리세린
핸드스크럽
필링
코코넛
파운데이션
SPF34
커버력
자연스러움
매트함
다크닝없음
자외선

풋스크럽
풋팩
풋스프레이
각질제거
아로마테라피
로즈마리
페퍼민트
유칼립투스
라벤더
베르가모트
라임
레몬그라스
바질
레몬
제라늄
아로마효과
그레이프프루트
마저럼
싸이프러스
일랑일랑
주니퍼베리
프랑킨센스
시더우드
클라리세이지
로즈
파인
시트로넬라
캐모마일
자스민
편백
만다린
샌달우드
타임
제모제
페이스
바디
팔/다리
겨드랑이
비키니라인
브라질리언
남자수염
헤어라인
왁스형
남녀공용
남성용
여성용
바디스크럽
사해소금
인그로운헤어
페이스겸용
쑥
바디오일
헤어겸용
목욕비누
바디솝
115g
때비누
100g
120g
170g
150g
140g
95g
250g
108g
90g
160g
105g
135g
130g
녹차
125g
75g
233g
110g
80g
106g
85g
60g
300g
200g
50g
350g
180g
오곡
15g
156g
1g
127.5g
94g
70g
10g
175g
138g
118g
142g
127g
141g
20g
165g
145g
198g
113g
126g
330g
190g
400g
240g
105gg
310g
107g
 94g
102g
225g
40g
182g
23g
122g
92g
97g
바디케어세트
핸드클렌저
데오드란트
땀흡수
샤워코롱
바디파우더
한방샴푸
중성
건조 모발용
컬러유지
패스트샴푸
건선
피부염성
드라이샴푸
아연샴푸
염색 모발용
트리트먼트
모발강화
단백질
LPP
케라틴
건선/피부염성
웨이브 모발용
프로폴리스
정전기방지
모발보호
모발탄력
헤어에센스
헤어오일
헤어세럼
헤어앰플
볼륨케어
헤어로션
헤어토닉
헤어미스트
새치케어
헤어팩
린스
컨디셔너
두피앰플
두피오일
두피팩
두피세럼
염색약
크림염색약
산화제
헤나염색약
두피영양
새치커버스틱
두피보습
버블염색약
탈색제
헤어쿠션
섀도우
헤어마스카라
헤어스프레이
세팅력
고정력
왁스
광택부여
손쉬운 세정
스프레이
하드
헤어왁스
크림왁스
포마드왁스
소프트
파마약
다운펌
곱슬 모발용
직모용
일반린스
퍼퓸린스
한방린스
보색린스
헤어케어세트
헤어젤
헤어무스
헤어글레이즈
클렌징파우더
보습
메이크업제거
스크럽
필

빌베리열매추출물
사탕수수추출물
설탕단풍추출물
세테아레스-20
해금사
라우라마이드디아에이
코카마이드디에이
솔비탄스테아레이트
청색205
소이아미도프로필아민옥사이드
다이-피피지-2미레스-10이디메이트
하이드록시프로필라이모늄하이드롤라이즈드일단백질
에틸실글리세린
아마란스피추출물
치마버섯균사체발효여과추출물
강낭콩씨추출물
들깨추출물
보리씨추출물
돌콩추출물
적색227호
알파-아이스아이오논
번질조에이트
시트르널올
핵실신남알
에칠헥실메독시신나메이트
육두구추출물
카프릴릴트리메치콘
비스-에칠헥실옥시페놀메독시페닐트리아진
이소아밀P-메톡시신나메이트
페닐벤즈이미다졸설포닉애씨드
폴리소르베이트20.동백꽃추출물
알펜로제추출물
맥문동추출물
오미자추출물
도인추출물
백출추출물
일당귀추출물
작약추출물
차전초추출물
상백피추출물
락토바실러스/배즙발효여과물
효모/보리씨발효여과물
락토바실러스/고추냉이뿌리발효추출물
락토바실러스/부레옥잠발효물
락토바실러스/인삼뿌리발효여과물
락토바실러스/콩발효추출물
락토바실러스/호밀가루발효물
붉은누룩곰팡이/쌀발효물
효모/감자추출물발효여과물
효모/자일리눔/홍차발발효물
효모/포도발효추출물
금은화추출물
락토바실러스/히비스커스꽃발효여과물
가시대나무수액
붉은토끼풀꽃추출물
감초추출물
강황추출물
개다래추출물
결명자추출물
고구마추출물
괄루근추출물
구기자추출물
귤껍질추출물
기장씨추출물
긴병꽃풀추출물
길경추출물
단풍나무잎추출물
당근추출물
더덕추출물
레몬밤추출물
레몬버베나잎추출물
마늘추출물
마치현추출물
모과추출물
무추출물
무화과추출물
바나나추출물
베르가모트잎추출물
비트뿌추출물
비파나무잎추출물
서양민들레잎추출물
선백리향추출물
세이지잎추출물
쇠뜨기추출물
수박추출물
순무잎추출물
시금치추출물
애엽추출물
양배추잎추출물
얼룩조릿대잎추출물
연꽃뿌리추출물
오이추출물
왕질경이씨추출물.원추리꽃추출물
유자추출물
으름덩굴추출물
은행추출물
일본산사나무열매추출물
잎새버섯자실체추출물
자두추출물
자소엽추출물
제주이질풀추출물
지모추출물
참미역추출물
창포추출물
치커리뿌리추출물
파파야열매추출물
팥추출물
페퍼민트꽃/잎/줄기추출물
표

트라이부틸시트레이트
효모/쌀발효여과물
커피콩추출물
5%)
폴리글리세릴-3스테아레이트
소듐카보머
마그네슘아스코빌포스페이트
달맞이꽃오일
아마씨오일
측백나무씨추출물
인삼뿌리세포추출물
하벌레아로도펜시스잎추출물
호프추출물
2-헥산다이몰
당근전초추출물
아이소헥사데칸
히더추출물
루이보스잎추출물
흑효모발효물
루틴
클라미도모나스추출물
키토산
폴리글리세릴-10아이소스테아레이트
아이소노닐아이소노나노에이트
호호바에스터
왕대추출물
프룩토올리고사카라이드
프룩토오스
하이드롤라이즈드글라이코사미노글리칸
신양벚나무꽃추출물
피피지-3카프릴릴에텔
피이지-14M
소듐자일렌설포네이트
적색228-7호
슈크로오스
C12-14알케스-12
2
3-부탄다이올
탱자추출물
흰목이버섯추출물
소나무잎추출물(81.0%)
칼라민
바오밥나무오일
브라시카미도프로필다이메틸아민
동백나무씨오일(1000ppm)
폴리라이신
귀리단백질추출물
락토바실러스/무뿌리발효여과물
효모/율무씨발효여과물
바실러스/하수오뿌리추출물발효여과물
바실러스/비타민나무열매발효추출물
바실러스/개구리밥추출물/화산재추출물발효여과물
락토바실러스/파파야열매발효추출물
락토바실러스/인삼열매/잎/뿌리추출물발효물
락토바실러스/약모밀잎추출발효여과물
락토바실러스/쌀겨발효여과물
락토바실러스/사탕수수/녹차발효여과추출물
락토바실러스/망고스틴?과피발효여과물
락토바실러스/마가목가지발효여과물
아크릴레이트/스테아레스-20메타크릴레이트코폴리머
락토바실러스/조류추출발효물
효모/슈가캠프발효물
알로에베라잎즙(90.52%)
아이리쉬모스추출물
라임전초오일
오렌지꽃오일
다이페닐실록시페닐트라이메티콘
다이메티콘/비닐다이메티콘크로스폴리머
아스코빅애씨드(5%)
하이드록시에칠셀룰로오스
팔각회향추출물
자몽추출물
브로콜리추출물
소듐아크릴레이트/소듐아크릴로일디메칠타우레이트코폴리머
녹차캘러스배양추출물
유럽잎갈나무추출물
세신추출물
신갈나무잎추출물
여뀌추출물
현호색추출물
황련추출물
후박추출물
라이신에이치씨엘
아세틸메치오닌
바실러스/콩/폴릭애씨드발효추출물
에델바이스캘러스배양추출물
잇꽃씨오일
한련초잎추출물
판테놀(10%)
폴

팔미토일펜타펩타이드-4(5ppb)
헥사펩타이드-9(5ppb)
소르비탄스테아레이트
에칠올리에이트
에틸스테아레이트
스클레로튬검
시어버터불검화물
에칠리놀리에이트
에틸팔미테이트
에델바이스꽃/잎추출물(0.0375%)
에델바이스캘러스배양추출물(0.0002%)
발삼아미리스껍질오일
쓴박하추출물바오밥나무씨추출물
향료(1.2%)
수레국화꽃수
마트리카리이꽃오일
하이드록시에틸아크릴레이트
소듐아크릴로일다이메틸타우레이트코폴리머
소듐스타치옥테닐석시네이트
무궁화나무껍질추출물
단향목부추출물
스노우로투스추출물
그나팔리움레온토포디움꽃추출물
●앰플:콜라겐추출물(86%)
다이소듐포스페이트●아보카도샷:아보카도열매추출물(30%)
마닐라엘레미트리검비휘발물
리날루리페룸녹나무목부오일
라우릴글루코사이
드라우릴베타인
소듐클로라이드글리세린
리날부틸페닐메칠프로피오날
폴리쿼터늄-10구아하이드록시프로필트리모늄클로라이드
고추열매추출물
구릿대뿌리추출물복분자딸기열매추출물
지치뿌리추출물하수오뿌리추출물
황금추출물영지버섯추출물산수유열매추출물
녹두전초추출물
종대황뿌리줄기/줄기추출물
천궁뿌리추출물참깨추출물
서양자두씨추출물
폴리쿼터늄-47
폴리우레탄-10
서양쐐기풀잎추출물
카룸페트로셀리눔추출물
자주개자리추출물
유채추출물
병풀추출물(85%)
붉은토끼풀추출물
콜라겐(11.2ppm)
테트라하이드로피페린
플로레틴
PPG-2하이드록시에틸코카마이드
다이소듐포스페이트●청귤샷:귤추출물(73%)
귤오일
코튼추출물
흰목이버섯폴리사카라이드(10670ppm)
C11-13아이소알케인
하이드로제네이티드올리브오일불검화물
소듐메타바이설파이트
보리잎추출물
베타카로틴1.331ng)
보검선인장열매수
라이코펜
유자씨오일
아비시니안케일씨오일
시카카이열매추출물
녹두씨추출물
와일드제라늄추출물
레이디스맨틀잎추출물
라임꽃추출물
천궁뿌리추출물
황기뿌리추출물
소듐코코일애플아미노산
당호박가루
사이클로메티콘
보검선인장열매수(920ppm)
맥주효모추출물(100000ppm)
라우레스-2폴리쿼터늄-10
아스파라진
하이드롤라이즈드헤이즐넛단백질
하이드롤라이즈드효모단백질
효모/콩단백질발효물
하

...

아데노신하이드록시에칠셀룰로오스
글리세릴스테아레이트디메치콘
마디풀추출물(75%)
부틸렌글아디콜
향료아민
페녹시에탄올(0.4%)
아까시나무꽃추출물(250ppm)
카프릴릭/카프릭트라이글리세라드
에칠파라벤프로필렌글라이콜
에톡시디글라이콜페녹시에탄올
메칠파라벤위치하젤추출물
글루코오스락틱애씨드
소듐하이드록사이드비에이치티
아스코빌팔미테이트글리세릴스테아레이트시트릭애씨드
다윈튤립꽃추출물
1/2-헥산다이올솔비탄스테아레이트
아크릴레이트/C10-3알킬아크릴레이트크로스폴리머
삼나무가지/잎추출물
C30-45알킬메티콘
C30-45올레핀
호두추출물
소디움라우레스설페이트
코카마이드이디이에이
노니과일추출물
라즈베리씨오일
리놀렌산
피이지-20글리세릴라우레이
디소디움이디티에이
마그네슘니트레이트
CI19140
CI42090
헥실신나말
코코넛야자씨버터
피이지-1001스테아레이트
퀴노아씨오일
그릴세릴스테아레이트에스이
소르비탄세스퀴올리에리트
조합향료
라벤더오일(10ppm)
육계껍질오일
다마스크장미추출물
테트라소듐이디테이
부틸렌글라이콜에틸헥실글리세린
풀솜대추출물
수레국화꽃추출물보리지추출물
라일락추출물
국화꽃추출물
노랑코스모스꽃/잎/줄기추출물
프리지어꽃추출물
코카미도프필베타인
라우라미노프로필렌글라이콜
코카인
포도씨오일(100ppm)
카미소니스아르니카꽃추출물(0.02%)
리우릭애씨드
시어버터(20000ppm)
향료(로즈앤스웨이드향0.75%)
사막장미잎세포추출물
동충하초추출물
복숭아추출물(1000ppm)
벤조페논-5
하이들롤라이즈드콜라겐
녹색401호
0-사이멘-5
올하이드록시아세토페논
미노넨
세테아릴알코을올리브오일
사이클로펜타실록세인사이클로헥사실록세인세테아릴올리베이트
옥탄다이을
하이드록시시트로넬알알파-아이소메틸아이오논시트랄
티트리추출물(5000ppm)
티트리잎오일(500ppm)
바나나펄프추출물
피이지150펜타에리스리틸테트라스테아레이트
솔비탄올리에이트데실글루코사이드크로스폴리머
세틸베타인
피이지6카프릴릭/카프릭글리세라이즈
시어버터아미도프로필베타인
소듐라우로일귀리아미노산
소듐아크릴레이트//소듐아크릴로일다이메틸타우레이

마그네슘팔미토일글루타메이트
폴리글리세릴-2트리이소스테아레이트
망가니즈바이올렛
다이페닐시롥시페닐트라이메티콘
라우릴피이지-10트리스(트라이메틸실록시)실릴에틸다이메티콘
아크릴레이트/스테아릴아크릴레이트/다이메티콘메타크릴레이트코폴리머
솔비탄세스퀴아이소스테아레이트
징크스테아레이트
프로폴리스추출물(0.2%)
회향추출물
자주천인국추출물
쟈스민꽃추출물
장뇌삼추출물
녹용추출물
디이소스테아릴말레이트
하이드로젠디메치콘
육계추출물
천문동추출물
익모초추출물
관동꽃추출물
칼슘스테아레이트
바실러스/콩발효추출물
라우릴피이지-8다이메티콘
폴리비닐알코올
티나늄옥사이트
피이지/피피지-18/18다이메티콘
보론나이트라이드
캐비어추출물
너도밤나무싹추출물
티타늄다이옥사이드
옥수수배아유
폴리글리세릴-3다이아소스테아레이트
비스머스옥시클로라이드
C20-22알킬포스페이트
타라열매추출물
코토니추출물
다이펩타이드다이아미노부티로일벤질아마이드다이아세테이트
트라이펩타이드-10시트룰린
C20-22알코올
정제수티타늄디옥사이드사이클로펜타실록세인호모살레이트에칠헥실살리실레이트세틸피이지/피피지-10/1다이메티콘트라이메틸실록시실리케이트글리세린카프릴릴메티콘부틸렌글라이콜변성알코올폴리메틸실세스퀴옥세인사이클로헥사실록세인아이소도데케인다이메티콘다이스테아다이모늄헥토라이트아크릴레이트/폴리트라이메틸실록시메타크릴레이트코폴리머황색산화철솔비탄세스퀴올리에이트마그네슘설페이트페녹시에탄올폴리글리세릴-4아이소스테아레이트다이메티콘/비닐다이메티콘크로스폴리머트라이에톡시카프릴릴실레인헥실라우레이트알루미늄하이드록사이드에틸헥실글리세린카프릴릴글라이콜적색산화철실리카다이메틸실릴레이트흑색산화철향료토코페릴아세테이트소듐하이알루로네이트판테놀비사보롤헥실신남알알파-아이소메틸아이오논펜타에리스리틸테트라-다이-t-부틸하이드록시하이드로신나메이트
메치콘
위치하젤꽃수
에칠멘탄카복사마이드
선인장꽃추출물
멘틸피씨에이
오크라열매추출물
우엉추출물
블랙멀베리잎추출물
폴리쿼터늄-61
명일엽잎/줄기추출물
디펜타에리스리틸헥사하이드록시스테아레이트/헥사스테아레이트/헥사로지네이트
세륨옥사이드
트리플루오로메칠C1-4알킬

비스-하이드록시에톡시프로필다이메티콘
윙스템패션플라워열매추출물
꽃생강뿌리추출물
인카르나타시계꽃전초추출물
메톡시피이지-100/폴리엡실론카프로락톤
마그네슘실리케이트
트라이데실트라이멜리테이트
마그네슘/포타슘실리콘플루오라이드하이드록사이드옥사이드
카올린진주가루
천문동뿌리추출물각시둥굴레뿌리줄기뿌리추출물하수오뿌리추출물
산수유열매추출물왜당귀뿌리추출물
홍삼추출물영지버섯추출물
둥굴레추출물
감국꽃추출물인동덩굴꽃추출물작약꽃추출물
적색산화철황색산화철
락토바실러스/콩방효추출물
소르비탄세스퀴올리레이트
메칠헥실글리세린
토타콜
팔각화향추출물
파파인
글리세릴트라이베헤네이트/아이소스테아레이트/에이코산디오에이트
말굽잔나비버섯추출물
포공영추출물
황백추출물
솔잣나무잎/넛트/줄기오일
아틀라스시더우드오일
아이소세테스-10
다이메티콘/페닐비닐다이메티콘크로스폴리머
마그네슘옥사이드
보론나이트라이트
에틸올리에이트
알루미늄다이미리스테이트
이소류신
이소말트
에스에이치-올리고펩타이드-9
효모/금발효용해여과물
아스파라거스추출물
헤마토코쿠스플루비알리스추출물
비닐다이메티콘/라우릴다이메티콘크로스폴리머
라우릴피이지-10트리스트라이메틸실록시)실릴에틸다이메티콘
비스-에칠핵실옥시페놀메톡시페닐트리아진
비스-하이드록시프로필다이메티콘/에스엠디아이코폴리머
글리세릴카프릴레이트카프릴릴글라이콜
트라이에톡시카프릴릴실레인미리스틱애씨드
비스머스옥시클로라이드하이드롤라이즈드엘라스틴
마그네슘포타슘플루오로실리케이트
실리카다이메티콘실릴레이트
콩스테롤
메도우폼에스톨라이드
글리세릴시트레이트/라테이트/리놀리에이트/올리에이트
알테로모나스발효여과물
향료03호정제수
향료04호정제수
귀리커넬추출물카렌둘라꽃추출물
아이브라이트추출물
마로니에잎추출물
글리세레스-20
디메치콘실릴레이트
피피지-12/에스엠디아이코폴리머
네오펜틸글라이콜디헵타노에이트
벤조트라이아졸릴도데실p-크레솔
트라이플루오로아세틸트라이펩타이드-2
덱스트란
INGLOTLOOSEPOWDER11탤크
카프릴릭/카프릭트라이글라이세라이드
수세미씨오일
폴리퍼플루오로메틸아이소프로필에터
INGLOTLOOSEPOWDER12탤크
I

수용성라스틴
글라이
노나이트
아사타이드-4카트라이타이드-1
트라이타이드-1
팔미토일테트라타이드
미토일트라이타이드-1
팔미토타이드사다이트
광산
산과산
소듐데히드로아세테이트
마돈나백합잎세포추출물
포타슘포스페이트
수련잎세포추출물
푸른연꽃잎세포추출물
소듐폴리글루타메이트크로스폴리머
플래티늄가루
진주층추출뭉
징크옥사이드.솔비탄세스퀴올리에이트
자이언트케프추출물
효모살레이트
오디추출물
베티-글루칸
하이드록시에칠피페라진에탄설포닉애씨드
폴리솔베이트40
아이소스테아릴하이드록시스테아레이트
하이드로제네이티드폴리(C6-20올레핀)
제인
솔잣나무가지/잎오일
칼슘옥사이드
벌독
피피지-2미리스틸에텔프로피오네이트
분홍바늘꽃추출물
피이지-20메칠글루코오스세스퀴스테아레이트
C12-15알킬에칠헥사노에이트
페녹시에탄올(ILN39205)
하이드로제네이티드C6-14올레핀폴리머
원지추출물
인도멀구슬나무추출물
니코티노일트리펩타이드-1
마그네슘/포타슘/실리콘/플루오라이드하이드록사이드옥사이드
사이클로헥산실록세인
세틸피이지/피피지
10/1다이메티콘
트라이에톡시
카프릴릴실레인
락토바실러스고추냉이뿌리발효추출물
효모포|도발효추출물
고수열매/잎추출물
구주물푸레잎추출물
블랙커민
씨추출물
한련꽃추출물
샤프란꽃추출물
락토바실러스부레옥잠발효물
모나스커스/쌀발효물
결명씨추출물
쿠네아타산사열매추출물
허브로버트추출물
기장추출물
파네실아세테이트
판테닐트리아세테이트
토코페롤아세테이트
피이지-8/에스엠디아이코폴리머
오수유추출물
슈크로오스팔미테이트
사이클로헥사시록산
황색산화철(ILN36644)
목화추출물
폴리소르베이80
라우르디모늄하이드록시프로필하이드롤라이즈드밀단백질
회화나무열매추출물
피이지-8카프릴릭/카프릭글리세라이드
시스틴
알지닌에이치씨엘
소듐피루베이트
올리고펩타이드-3
올리고펩타이드-4
파에오놀
황생산화철
영실추출물
세틸카프릴레이트
징크리미스테이트
다이메틸이미다졸리다이논쌀전분
유제놀탤크
옥도크릴렌
비스-에칠헥실옥시페놀메톡
페닐트리아진
동백꽃추출물에델바이스꽃/잎추출물
류코노스/무발효여과물
트리에독시카프릴릴실란
판도락톤
하이드롤라이즈드

스테아랄코늄벤토나이트
사이클로펜타실록제인
다이메티콘비날다이메티콘크로스폴리머
스하이알루로네이트
소듐아크릴레이트소품아크릴로일다이메틸타우레이트코폴리머
잔나비버섯추출물
베르가모트오일오렌지껍질오일
라임전조오일
다마스크장미수옥수수전분
페퍼민트임추출물
아이소사데칸
풀라솔베이트60.솔비탄올리에이트트라이에톡시카프릴설레인
징크억사이드
피이지-10다이메치콘
캐비어수(41.9615%)
헤스페리딘메틸칼콘
다이펩타이드-2
클로헥시딘다이글루코네이트
자일로오스
흰무늬엉겅퀴열매추출물
캐비아추출물(0.0004%)
트라이메틸올프로판트라이아이소스테아레이트
계혈등줄기추출물
다이소듐석시네이트
피이지-240/에이치디아이코폴리머비스-데실테트라데세스-20에터
징크라우레이트
하이드
중국목련나무꽃추출물
백합꽃/잎/줄기추출물
폴리아미노슈가축합물
이소프로필라놀레이트
프로필렌글라이콜스테아레이트
소듐콘드로이틴설페이트
실리카디메칠실리레이트
글리세릴트레아레이트
글라이콜팔미네이트
메타메칠크릴레이트크로스폴리머
실레네우니플로라추출물
레이트
트리이소스테아레이트
메치콘코폴리머
폴리글리세린-2
폴리메칠에타크릴
2-헥산다이올.부틸렌글라이콜
라벤더오일페퍼민트오일/로즈마리잎오일/카프릴릭애씨드카프릭트리글리세라이드살구스위트아몬드오일/호호바오일/당근씨오일/화분추출물
백리향추출물
오조케라이
곤약뿌리추출물
트리에톡시시카프릴릴실란
디메치콘올스테아레이드
아미노프로필트리에톡시실란
폴리아크릴레이트-13.스테아릭애씨드
폴리소르베이트20.마그네슘알루미늄실리케이트
위치하젤껍질//잔가지추출물
C30-45알킬디메칠실릴폴리프로필실세스퀴옥산
유향유
헥사데케인
테트라데케인
에이코세인
녹차잎추출물
씨드릭애씨드
비아이에스-피이지/피피지-14/14디메치콘
디헥실데실라우로일글루타메이트
PEG-9폴리디메칠실록시에칠다이메치콘
서양수수꽃다리잎세포배양추출물
울산도깨비바늘추출물
레시틴/아시아티코사이드/아시아틱애씨드/마데카식애씨드
브델리아분열조직세포배양물
애엽수
알지닉애씨드
플루란
캐모마일잎추출물
숙근천인국추출물
참으아리추출물
폴리글리세릴-10
라우레이트
피이지-240/에이치디아이